### Import Required Libraries and Set Up Environment Variables

In [37]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv("NASA_API_KEY")

### CME Data

In [38]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
end_date   = "2024-05-01"

# Build URL for CME
cme_query_url = f"{base_url}{CME}?startDate={startDate}&endDate={end_date}&api_key={NASA_API_KEY}"


In [ ]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(cme_query_url)
print(cme_response)

In [ ]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()


In [ ]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data

print(json.dumps(cme_json[0], indent=4))


In [ ]:
# Convert cme_json to a Pandas DataFrame  
cme = pd.DataFrame(cme_json)
cme.head()

In [ ]:
# Keep only the activityID, startTime, linkedEvents columns

cme = cme[['activityID', 'startTime', 'linkedEvents']]
cme.head()

In [ ]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme = cme.dropna(subset=['linkedEvents'])
cme.head()

In [ ]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for i in cme.index:
      
    activityID       = cme.loc[i, 'activityID']    # Get the corresponding value from 'activityID'
    startTime        = cme.loc[i, 'startTime']     # Get the corresponding value from 'startTime'    
    linkedEvents     = cme.loc[i, 'linkedEvents']  # Get the list of dictionaries in 'linkedEvents'

    try:

        # Iterate over each dictionary in the list
        for item in linkedEvents:
    
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
         expanded_rows.append({'activityID': activityID, 'startTime': startTime, 'linkedEvents': item})
    except:
        print('missing activity')
        
# Create a new DataFrame from the expanded rows
cme_for_loop = pd.DataFrame(expanded_rows)
cme_for_loop.head()

In [ ]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors

def extract_activityID_from_dict(input_dict):

    try:
        # Attempt to extract the 'activityID' from the input dictionary
        return input_dict['activityID']

    except (ValueError, TypeError) as e:
        # Handle the case where 'activityID' is not found or input is not a dict
        return None  

        # Log the error or print it for debugging

        return

extract_activityID_from_dict(cme_for_loop.loc[0,'linkedEvents'])


In [ ]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
cme['GST_ActivityID'] = cme['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x) if isinstance(x, dict) else None)
cme.loc[:, 'GST_ActivityID'] = cme['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x) if isinstance(x, dict) else None)
print(cme[['linkedEvents', 'GST_ActivityID']])

In [ ]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
cme = cme.dropna(subset=['GST_ActivityID'])
print(cme[['linkedEvents', 'GST_ActivityID']])

In [ ]:
# print out the datatype of each column in this DataFrame:
print(cme.dtypes)


In [ ]:
# Convert the 'GST_ActivityID' column to string format 
cme['GST_ActivityID'] = cme['GST_ActivityID'].astype(str)


# Convert startTime to datetime format  
cme['startTime'] = cme.to_datetime(cme['startTime'])

# Rename startTime to startTime_CME and activityID to cmeID
cme.rename(columns={'startTime': 'startTime_CME', 'activityID': 'cmeID'}, inplace=True)

# Drop linkedEvents
cme.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly
# 1. Check column names
print("Column names:", cme.columns)

# 2. Check data types
print("Data types:\n", cme.dtypes)

# 3. Check if 'linkedEvents' column is dropped
if 'linkedEvents' not in cme.columns:
    print("'linkedEvents' column has been dropped successfully.")
else:
    print("'linkedEvents' column is still present.")


In [ ]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
cme = cme[cme['GST_ActivityID'].str.contains('GST', na=False)]
print(cme)

### GST Data

In [ ]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
end_date   = "2024-05-01"

# Build URL for GST
gst_query_url = f"{base_url}{GST}?startDate={startDate}&endDate={end_date}&api_key={NASA_API_KEY}"


In [ ]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_query_url)
print(gst_response)

In [ ]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()

In [ ]:
# Convert the response variable to json and store it as a variable named gst_json
# Format the first result in JSON with indentation for readability
print(json.dumps(gst_json[0], indent=4))

In [ ]:
# Convert gst_json to a Pandas DataFrame  
gst = pd.DataFrame(gst_json)
gst.head()

In [ ]:
# Keep only the columns: activityID, startTime, linkedEvents
gst = gst[['gstID', 'startTime', 'linkedEvents']]
gst.head()

In [ ]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst = gst.dropna(subset=['linkedEvents'])
gst.head()

In [ ]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_exploded = gst.explode('linkedEvents')

# Reset the index
gst_exploded.reset_index(drop=True, inplace=True)

# Drop missing values
gst_exploded.dropna(subset=['linkedEvents'], inplace=True)

# Display the resulting DataFrame
print(gst_exploded)

In [ ]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
def extract_activityID_from_dict(input_dict):
    try:
        return input_dict['activityID']  # Adjust this according to your dictionary structure
    except (ValueError, TypeError) as e:
        return None  # Return None if there's an error

gst['CME_ActivityID'] = gst['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))
print(gst)


# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst.dropna(subset=['CME_ActivityID'], inplace=True)
print(gst)

In [ ]:
# Convert the 'gstID' column to string format 
gst['gstID'] = gst['gstID'].astype(str)


# Convert startTime to datetime format  
gst['startTime'] = pd.to_datetime(gst['startTime'])


# Rename startTime to startTime_GST 
gst.rename(columns={'startTime': 'startTime_GST'}, inplace=True)


# Drop linkedEvents
gst.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly
# 1. Check column names
print("Column names:", gst.columns)

# 2. Check data types
print("Data types:\n", gst.dtypes)

# 3. Check if 'linkedEvents' column is dropped
if 'linkedEvents' not in gst.columns:
    print("'linkedEvents' column has been dropped successfully.")
else:
    print("'linkedEvents' column is still present.")

In [ ]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
gst = gst[gst['CME_ActivityID'].str.contains('CME', na=False)]
print(gst)

### Merge both datatsets

In [ ]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(gst, cme, 
                     left_on=['gstID', 'CME_ActivityID'], 
                     right_on=['GST_ActivityID', 'cmeID'], 
                     how='outer')

print(merged_df.shape)

In [ ]:
# Verify that the new DataFrame has the same number of rows as cme and gst
# Check the number of rows in the original DataFrames
cme_rows = cme.shape[0]
gst_rows = gst.shape[0]

# Check the number of rows in the merged DataFrame
merged_rows = merged_df.shape[0]

# Print the results
print(f"Number of rows in cme: {cme_rows}")
print(f"Number of rows in gst: {gst_rows}")
print(f"Number of rows in merged DataFrame: {merged_rows}")

# Verify if the number of rows in the merged DataFrame matches the original DataFrames
if merged_rows == cme_rows + gst_rows:
    print("The merged DataFrame has the expected number of rows.")
else:
    print("The merged DataFrame does not have the expected number of rows.")

### Computing the time it takes for a CME to cause a GST

In [ ]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df['startTime_gst'] = pd.to_datetime(df['startTime_gst'])
merged_df['startTime_cme'] = pd.to_datetime(df['startTime_cme'])

merged_df['timeDiff'] = merged_df['startTime_gst'] - merged_df['startTime_cme']
print(merged_df[['startTime_gst', 'startTime_cme', 'timeDiff']].head())


In [ ]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
# Use describe() to compute summary statistics
time_diff_stats = merged_df['timeDiff'].describe()

# To specifically get the mean and median
mean_time_diff = time_diff_stats['mean']
median_time_diff = time_diff_stats['50%']  # The 50th percentile is the median

print(f"Mean time difference: {mean_time_diff}")
print(f"Median time difference: {median_time_diff}")

### Exporting data in csv format

In [ ]:
# Export data to CSV without the index
merged_df.to_csv("output_file.csv", index=False)